In [14]:
import pandas as pd
import numpy as np
# Visualizations
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
%matplotlib inline
from pandas.plotting import scatter_matrix

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score, r2_score
from sklearn import svm
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance

In [4]:
##HELPER METHODS
def featuresprocess(df):
    '''
    Use to map features of df 
      usually called for both df_train and df_test to line them up
    '''
    resdf = df.copy()
    resdf = resdf.fillna('Unknown')
#     resdf.drop(['gsm'], axis = 1, inplace = True)
    resdf.drop(['page_id'], axis = 1, inplace = True)
    resdf.drop(['urlslug'], axis = 1, inplace = True)
    resdf.drop(['first appearance'], axis = 1, inplace = True)
    resdf.drop(['id'], axis = 1, inplace = True)
    resdf.drop(['align'], axis = 1, inplace = True)
    resdf.drop(['name'], axis = 1, inplace = True)
    resdf.drop(['year'],axis = 1, inplace = True)
#   resdf.drop(['education'], axis = 1, inplace = True)
#   resdf.drop(['sex'], axis = 1, inplace = True)
  # resdf.drop(['native.country'],axis=1, inplace=True)
  # resdf.drop(['fnlwgt'],axis=1, inplace=True)
  # resdf.drop(['education.num'], axis = 1, inplace = True)
  # resdf['education'].replace(['11th', '9th', '7th-8th', '5th-6th', '10th', '1st-4th', 'Preschool', '12th'],
                              # 'K12School', inplace = True)
  # resdf['marital.status'] = resdf['marital.status'].replace(['Widowed', 'Divorced', 'Separated', 'Never-married'], 'single')
  # resdf['marital.status'] = resdf['marital.status'].replace(['Married-spouse-absent', 'Married-civ-spouse', 'Married-AF-spouse'], 'married')
  # resdf['relationship']=resdf['relationship'].replace(['Husband','Wife'],'Couple')
  # resdf['relationship'] = resdf['relationship'].replace(['Unmarried', 'Not-in-family','Own-child','Other-relative'], 'Others')
    return resdf

def split_XYtrain_data(df, test_size=0.2, random_state=0):
  '''Only use for train data (with income>50k label)
  Return Xtrain, Xtest, Ytrain, Ytest
  '''
  X = df.drop(labels=['appearances'],axis=1)
  Y = df['appearances']
  return  train_test_split(X, Y, test_size =test_size, random_state = random_state)

def encode_features(df_train, df_test, onlyone=False):
  '''
  Encode labels for two X data sets
  -If use for split test and train. Pass in Xtrain and Xtest
  -If use for real test and full train. Pass in full X data train, Xtest_kaggle
  '''
  df_combined = pd.concat([df_train, df_test])
  categorical = [column for column in df_combined if df_combined.dtypes[column] == np.object]
  # categorical = ['workclass','education', 'marital.status', 'occupation', 'relationship','race', 'sex']#drop native.country
  for feature in categorical:
    encoder = preprocessing.LabelEncoder()
    encoder = encoder.fit(df_combined[feature])
    df_train[feature] = encoder.transform(df_train[feature])
    if not onlyone:
      df_test[feature] = encoder.transform(df_test[feature])
  return df_train, df_test

def crossvaldata_sklearn(df, split=0.3,rdstate=0):
  """Return preprocessed data, split into train and test
  (not used when apply to kaggle data)
  """
  df = featuresprocess(df)
  Xtrain, Xtest, Ytrain, Ytest = split_XYtrain_data(df, test_size=split, random_state=rdstate)
  Xtrain, Xtest = encode_features(Xtrain, Xtest)
  return Xtrain, Xtest, Ytrain, Ytest


In [10]:
df_train = pd.read_csv('https://raw.githubusercontent.com/baohuy251210/ComicCharactersMining/main/data/wikia/marvel-wikia-data.csv',na_values='?')


In [11]:
print("Missing values:")
df = df_train
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum()/df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])


##For report only
with open('./train_final_missingvalues.tex', 'w') as tf:
        tf.write(missing_data.to_latex(index=True, float_format="%1.2f", label='tab:missingvals',
                                    caption="Missing values from Train data"))
missing_data.head(5)

Missing values:


,Total,%
gsm,16286,99.5
eye,9767,59.6
hair,4264,26.0
id,3770,23.0
align,2812,17.2


In [12]:
#drop gsm, replace others with "unknown"
# df.drop(['gsm'], axis = 1, inplace = True)
# df.fillna('Unknown', inplace=True)




In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent_1 = df.isnull().sum()/df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.to_csv("marvel_mostcm.csv", index=False)

In [5]:
df = pd.read_csv("marvel_mostcm2.csv")
# df = df.dropna(subset=['appearances','year'])
# df.to_csv("marvel_mostcm2.csv", index=False)
# df.fillna()
scaler = StandardScaler()

from sklearn.metrics import classification_report, mean_squared_error

xtrain, xtest, ytrain, ytest = crossvaldata_sklearn(df)
scalecols = [x for x in xtrain.columns]
# xtrain = pd.DataFrame(scaler.fit_transform(xtrain), columns = scalecols)

# xtest = pd.DataFrame(scaler.transform(xtest), columns = scalecols)
print(xtrain.head(5))

random_forest =RandomForestRegressor(n_estimators=50, max_depth=3)
random_forest.fit(xtrain, ytrain)
predict_rf = random_forest.predict(xtest)
acc_rf = random_forest.score(xtest, ytest)*100
acc_rf_t = random_forest.score(xtrain, ytrain)*100

print("Sklearn random forest\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(acc_rf_t,acc_rf))
print("MSE on TRAIN: ",mean_squared_error(ytrain, random_forest.predict(xtrain)))
print("MSE on TEST: ",mean_squared_error(ytest, predict_rf))

       eye  hair  sex  alive
12132    2     2    1      1
5607     7     6    1      1
1942    19     6    1      1
10415   19    22    3      1
5438    19    22    3      1
Sklearn random forest
Train: 4.02339
Test: 4.12528
MSE on TRAIN:  10488.041075583396
MSE on TEST:  5733.789231625607


In [6]:
adaboost =AdaBoostRegressor(n_estimators=100)
adaboost.fit(xtrain, ytrain)
predict_ada = adaboost.predict(xtest)
acc_ada = adaboost.score(xtest, ytest)*100
acc_ada_t = adaboost.score(xtrain, ytrain)*100

print("Sklearn Adaboost forest\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(acc_ada_t,acc_ada))
print("MSE on TRAIN: ",mean_squared_error(ytrain, adaboost.predict(xtrain)))
print("MSE on TEST: ",mean_squared_error(ytest, predict_ada))

Sklearn Adaboost forest
Train: -109.16025
Test: -226.11430
MSE on TRAIN:  22856.416640635878
MSE on TEST:  19503.271578875774


In [7]:
gdr =GradientBoostingRegressor(n_estimators=200)
gdr.fit(xtrain, ytrain)
predict_gdr  = gdr.predict(xtest)
acc_gdr  = gdr.score(xtest, ytest)*100
acc_gdr_t = gdr.score(xtrain, ytrain)*100

print("Sklearn Gradient forest\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(acc_gdr_t,acc_gdr))
print("MSE on TRAIN: ",mean_squared_error(ytrain, gdr.predict(xtrain)))
print("MSE on TEST: ",mean_squared_error(ytest, predict_gdr))

Sklearn Gradient forest
Train: 6.82721
Test: 4.74025
MSE on TRAIN:  10181.647601365205
MSE on TEST:  5697.010796793664


In [8]:
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
logr = LogisticRegression(max_iter=1000)
logr.fit(xtrain, ytrain)
lg_test = logr.score(xtest, ytest)
lg_train = logr.score(xtrain, ytrain)
print("Sklearn Log R\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(lg_test,lg_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, logr.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, logr.predict(xtest)))

Sklearn Log R
Train: 0.31182
Test: 0.31145
MSE TRAIN:  11212.195421334633
MSE TEST:  6210.8720454545455


c:\users\baohu\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
linearreg = LinearRegression()
linearreg.fit(xtrain, ytrain)
linearreg_test = linearreg.score(xtest, ytest)
linearreg_train = linearreg.score(xtrain, ytrain)
print("Sklearn Linear Regression\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(linearreg_test,linearreg_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, linearreg.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, linearreg.predict(xtest)))

Sklearn Linear Regression
Train: 0.03201
Test: 0.02615
MSE TRAIN:  10641.967731275114
MSE TEST:  5789.057227573969


In [10]:
rr = Ridge(alpha=0.01)
rr.fit(xtrain, ytrain)
rr_test = rr.score(xtest, ytest)
rr_train = rr.score(xtrain, ytrain)
print("Sklearn Ridge Regression\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(rr_test,rr_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, rr.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, rr.predict(xtest)))

Sklearn Ridge Regression
Train: 0.03201
Test: 0.02615
MSE TRAIN:  10641.96773127557
MSE TEST:  5789.057199368543


In [11]:
lasso = Lasso(alpha=0.02)
lasso.fit(xtrain, ytrain)
lasso_test = lasso.score(xtest, ytest)
lasso_train = lasso.score(xtrain, ytrain)
print("Sklearn Lasso Regression\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(lasso_test,lasso_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, lasso.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, lasso.predict(xtest)))

Sklearn Lasso Regression
Train: 0.03204
Test: 0.02615
MSE TRAIN:  10641.970604808852
MSE TEST:  5788.899055519441


In [12]:
from sklearn.linear_model import BayesianRidge
brr = BayesianRidge()
brr.fit(xtrain, ytrain)
brr_test = brr.score(xtest, ytest)
brr_train = brr.score(xtrain, ytrain)
print("Sklearn Bayesian Ridge Regression\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(brr_test,brr_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, brr.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, brr.predict(xtest)))

Sklearn Bayesian Ridge Regression
Train: 0.03204
Test: 0.02601
MSE TRAIN:  10643.474851602381
MSE TEST:  5788.882332925896


In [13]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=6)
knr.fit(xtrain, ytrain)
knr_test = knr.score(xtest, ytest)
knr_train = knr.score(xtrain, ytrain)
print("Sklearn KNeighborsRegressor\nTrain: {0:5.5f}\nTest: {1:5.5f}".format(knr_test,knr_train))

print("MSE TRAIN: ",mean_squared_error(ytrain, knr.predict(xtrain)))
print("MSE TEST: ",mean_squared_error(ytest, knr.predict(xtest)))

Sklearn KNeighborsRegressor
Train: -0.01864
Test: 0.01373
MSE TRAIN:  10777.72054446068
MSE TEST:  6091.961414141415
